In [3]:
# we will use 5 different strategies (outlined by Schwab research)
    # Perfect market timing, investing $2,000 once a year at the lowest point.
    # Investing $2,000 per year on the first trading day.
    # Dividing $2,000 into 12 pieces and investing at the beginning of every month.
    # The opposite of number one, investing $2,000 at the highest point.
    # Left money in cash only, no investments.

# we will also check what happens if you skipped making a reocurring investment during the best 10,20 weeks/months of a given period


In [61]:
# libraries 
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import datetime
import random

# Data Collection

In [142]:
rfr=yf.download('^IRX',period='max',interval='1d')

[*********************100%***********************]  1 of 1 completed


In [145]:
rfr_annual=rfr.resample('Y').mean()

In [6]:
sp=yf.download('^GSPC',period='max',interval='1d')

[*********************100%***********************]  1 of 1 completed


In [112]:
sp_close=sp.loc[:,['Close']]
weekly_sp=sp_close.resample('W').last()
monthly_sp=sp_close.resample('M').first()

In [113]:
sp_close['Return']=sp_close.pct_change()
weekly_sp['Return']=weekly_sp.pct_change()
monthly_sp['Return']=monthly_sp.pct_change()

# Strategy 1- Perfect Investor

In [138]:
def min_year(years,start_year,invest_amt):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    for i in range(years):
        min_date=sp_close.loc[datetime.date(start_year,1,1)+relativedelta(years=i):datetime.date(start_year,12,31)+relativedelta(years=i),'Close'].idxmin()
        returns=invest_amt*((1+sp_close.loc[min_date-relativedelta(days=1):datetime.date(start_year,12,31)+relativedelta(years=10),'Return']).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def min_year_multiple_tests(tests,years,invest_amt):
    final_returns=[]
    while tests>0:
        #getting a random year that is within the data range that fits the timeline we need 
        start_year=random.randint(sp_close.index.min().year,sp_close.index.max().year-years)
        final_returns.append(min_year(years,start_year,invest_amt)[-1])
        tests-=1
    return final_returns
        
def min_year_multiple_tests_chart(tests,years,invest_amt):
    fig=go.Figure()
    fig.add_trace(go.Box(name='Returns',y=min_year_multiple_tests(tests,years,invest_amt)))
    fig.show()

In [139]:
min_year_multiple_tests_chart(15,10,2000)


# Strategy 2- Investing on first trading day

In [260]:
def first_trading_day(years,start_year,invest_amt):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    for i in range(years):
        returns=invest_amt*((1+sp_close.loc[datetime.date(start_year,1,1)+relativedelta(years=i):datetime.date(start_year,12,31)+relativedelta(years=i),'Return']).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def first_trading_day_multiple_tests(tests,years,invest_amt):
    final_returns=[]
    while tests>0:
        #getting a random year that is within the data range that fits the timeline we need 
        start_year=random.randint(sp_close.index.min().year,sp_close.index.max().year-years)
        final_returns.append(first_trading_day(years,start_year,invest_amt)[-1])
        tests-=1
    return final_returns

def first_trading_day_multiple_tests_chart(tests,years,invest_amt):
    fig=go.Figure()
    fig.add_trace(go.Box(name='Returns',y=first_trading_day_multiple_tests(tests,years,invest_amt)))
    fig.show()

In [132]:
first_trading_day_multiple_tests_chart(10,10,1000)


# Strategy 3- Investing equal amounts at the beginning of every month

In [261]:
def first_day_month(years,start_year,invest_amt):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    for i in range(years):
        for m in range(12):
            returns=invest_amt/12*((1+monthly_sp.loc[datetime.date(start_year,1,1)+relativedelta(years=i,months=m):datetime.date(start_year,12,31)+relativedelta(years=i),'Return']).cumprod()[-1])
            total_returns+=returns
            total_invested+=invest_amt/12
            profit_loss+=returns-invest_amt/12
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def first_day_month_multiple_tests(tests,years,invest_amt):
    final_returns=[]
    while tests>0:
        #getting a random year that is within the data range that fits the timeline we need 
        start_year=random.randint(sp_close.index.min().year,sp_close.index.max().year-years)
        final_returns.append(first_day_month(years,start_year,invest_amt)[-1])
        tests-=1
    return final_returns

def first_day_month_multiple_tests_chart(tests,years,invest_amt):
    fig=go.Figure()
    fig.add_trace(go.Box(name='Returns',y=first_day_month_multiple_tests(tests,years,invest_amt)))
    fig.show()

In [217]:
first_day_month_multiple_tests_chart(10,10,1000)

# Strategy 4- Investing at highest point

In [180]:
def max_year(years,start_year,invest_amt):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    for i in range(years):
        max_date=sp_close.loc[datetime.date(start_year,1,1)+relativedelta(years=i):datetime.date(start_year,12,31)+relativedelta(years=i),'Close'].idxmax()
        returns=invest_amt*((1+sp_close.loc[max_date-relativedelta(days=1):datetime.date(start_year,12,31)+relativedelta(years=10),'Return']).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def max_year_multiple_tests(tests,years,invest_amt):
    final_returns=[]
    while tests>0:
        #getting a random year that is within the data range that fits the timeline we need 
        start_year=random.randint(sp_close.index.min().year,sp_close.index.max().year-years)
        final_returns.append(max_year(years,start_year,invest_amt)[-1])
        tests-=1
    return final_returns
        
def max_year_multiple_tests_chart(tests,years,invest_amt):
    fig=go.Figure()
    fig.add_trace(go.Box(name='Returns',y=max_year_multiple_tests(tests,years,invest_amt)))
    fig.show()

In [141]:
min_year_multiple_tests_chart(10,10,1000)

# Strategy 5- Left money in 3M Treasury, no investments

In [183]:
# we will use the 13W Treasury as risk free rate 

In [262]:
def no_invest(years,start_year,invest_amt):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    for i in range(years):
        returns=invest_amt*((1+rfr_annual.loc[datetime.date(start_year,1,1):datetime.date(start_year,12,31)+relativedelta(years=i),'Close']/100).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def no_invest_multiple_tests(tests,years,invest_amt):
    final_returns=[]
    while tests>0:
        #getting a random year that is within the data range that fits the timeline we need 
        start_year=random.randint(rfr_annual.index.min().year,rfr_annual.index.max().year-years)
        final_returns.append(no_invest(years,start_year,invest_amt)[-1])
        tests-=1
    return final_returns

def no_invest_multiple_tests_chart(tests,years,invest_amt):
    fig=go.Figure()
    fig.add_trace(go.Box(name='Returns',y=no_invest_multiple_tests(tests,years,invest_amt)))
    fig.show()

In [175]:
no_invest_multiple_tests_chart(10,10,1000)

# Comparing Strategies

In [218]:
def all_tests(tests,years,invest_amt):
    final_returns=[]
    while tests>0:
        #getting a random year that is within the data range that fits the timeline we need 
        start_year=random.randint(rfr_annual.index.min().year,rfr_annual.index.max().year-years)
        final_returns.append([min_year(years,start_year,invest_amt)[-1],first_trading_day(years,start_year,invest_amt)[-1],first_day_month(years,start_year,invest_amt)[-1],max_year(years,start_year,invest_amt)[-1],no_invest(years,start_year,invest_amt)[-1]])
        tests-=1
    return final_returns

def no_invest_multiple_tests_chart(tests,years,invest_amt):
    fig=go.Figure()
    fig.add_trace(go.Box(name='Returns',y=no_invest_multiple_tests(tests,years,invest_amt)))
    fig.show()

In [232]:
def all_strategies(tests,years,invest_amt):
    all_tests(tests,years,invest_amt)
    fig=go.Figure()
    fig.add_trace(go.Box(boxmean=True,name='Worst Timing',y=[x[i][3] for i in range(years)]))
    fig.add_trace(go.Box(boxmean=True,name='1st Trade Day',y=[x[i][1] for i in range(years)]))
    fig.add_trace(go.Box(boxmean=True,name='1st Day Month',y=[x[i][2] for i in range(years)]))
    fig.add_trace(go.Box(boxmean=True,name='Perfect Timing',y=[x[i][0] for i in range(years)]))
    fig.add_trace(go.Box(boxmean=True,name='No Invest',y=[x[i][4] for i in range(years)]))
    fig.update_layout(boxmode='group')    
    fig.show()
    
all_strategies(100,5,1000)

In [235]:
#several conclusions
#perfect timing is undeniably great, but unlikely to achieve even for Warren Buffet 
#The other investing methods are not so far behind. 
#investing in the 3M treasury actually gives limited returns, but if we compare it with the worst timing, its actually outperforming significantly.
#worst timing is just as likely as the perfect timing however.
#investing on the 1st trade day vs every month is a bit tricky- its the idea of the dollar cost vs lump sum. 
#if the period for investment is longer, investing the money right away is giving slightly higher returns vs investing in chunks
#overall in the 5 investment periods, you're not likely to experience a loss (the only exception is if you have the worst timing)


# What if you had reocurring investment but missed the top 10,20 weeks/months

In [293]:
#one example is covid- an investor thought the world is coming to an end so pulled out the market, fearing greater selloff.
#instead of stocks plummeting further, market bounced back and the investor missed on the bounce.
#also what if the investor was smart and managed to get out of the market before the worst selloffs.

In [302]:
def miss_worst_days(years,start_year,invest_amt,days):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    top_days=weekly_sp.loc[datetime.date(start_year,1,1):datetime.date(start_year,12,31)+relativedelta(years=years)].sort_values(by='Return',ascending=True).head(days).index
    data=weekly_sp.loc[datetime.date(start_year,1,1):datetime.date(start_year,12,31)+relativedelta(years=years)].drop(top_days)
    for i in range(len(data)): 
        returns=invest_amt*((1+data.iloc[i:]['Return']).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def miss_best_days(years,start_year,invest_amt,days):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    top_days=weekly_sp.loc[datetime.date(start_year,1,1):datetime.date(start_year,12,31)+relativedelta(years=years)].sort_values(by='Return',ascending=False).head(days).index
    data=weekly_sp.loc[datetime.date(start_year,1,1):datetime.date(start_year,12,31)+relativedelta(years=years)].drop(top_days)
    for i in range(len(data)): 
        returns=invest_amt*((1+data.iloc[i:]['Return']).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct

def no_miss(years,start_year,invest_amt):
    total_returns=0
    total_invested=0
    profit_loss=0
    profit_loss_pct=0
    data=weekly_sp.loc[datetime.date(start_year,1,1):datetime.date(start_year,12,31)+relativedelta(years=years)]
    for i in range(len(data)): 
        returns=invest_amt*((1+data.iloc[i:]['Return']).cumprod()[-1])
        total_returns+=returns
        total_invested+=invest_amt
        profit_loss+=returns-invest_amt
    profit_loss_pct=(total_returns-total_invested)/total_invested
    return total_returns,total_invested,profit_loss,profit_loss_pct


def comparison(years,start_year,invest_amt,days):
    total=[]
    total.extend([miss_worst_days(years,start_year,invest_amt,days),miss_best_days(years,start_year,invest_amt,days),no_miss(years,start_year,invest_amt)])
    return total

In [332]:
def miss_chart(years,start_year,invest_amt,days):
    fig=go.Figure()
    fig.add_trace(go.Bar(text=[round(i[-1],2) for i in comparison(years,start_year,invest_amt,days)],x=['Worst','Best','No Miss'],y=[i[-1] for i in comparison(years,start_year,invest_amt,days)]))
    fig.update_layout(title=f'Missing top {days} between {datetime.date(start_year,1,1)} - {datetime.date(years+start_year,12,31)} ')
    fig.show()

In [333]:
miss_chart(1,2020,100,10)
#we all know 2020 was a volatile year, but the difference in the performance is staggering!